## We are taking prediction results of a district, taken at different point of time (different months within a year). We are going to combine all prediction results to make a final prediction for that year. 

In [1]:
# import all required packsges
from PIL import Image
import math
from PIL import Image
from scipy import misc
from scipy import ndimage
import imageio
import pandas as pd
import unittest
import os, sys
import numpy as np

from collections import Counter

ModuleNotFoundError: No module named 'scipy'

In [ ]:
# Covert monthly predictions from .tif to .png


cities = ['Gurgaon','Hyderabad','Mumbai']
for district in cities:
    main_folder=district     #we have stored monthly predections in a folder named by district name
    print(district)
    os.makedirs(main_folder+"/pngs",exist_ok=True)
    for infile in os.listdir(main_folder):
        if infile[-4:] == ".tif":                   #reading all tif files in given folder
            im = Image.open(main_folder+"/"+infile)
            im.save(main_folder+"/pngs/"+infile[:-4]+'.png')

In [19]:
# Defining all required functions

def min(a,b):    #find minimum between a and b
    if a<b:
        return a
    return b

def is_number_exist(num,listx):   #membership test
    listx=set(listx)
    if num in listx:
        return True
    
def count(num,listx):                       #count the occurence of 'num' in 'listx'
    keys = list(Counter(listx).keys())
    values = list(Counter(listx).values())
    flag = False
    flag = is_number_exist(num,keys)
    if flag == True:
        num_count = values[keys.index(num)]
    else:
        num_count = 0
    return num_count


  
#merge the monthly predictions and overall median prediction to get a single prediction for year
# 0 denotes Background, 1 denotes greenery, 2 denotes water, 3 benotes Built-Up, 4 denotes Bareland
def merge_prediction(x,median):
    
    #find the count of each kind of pixel value for a pixel across all the given years
    background_count = count(0,x)
    grass_count = count(1,x)
    water_count = count(2,x)
    buildings_count = count(3,x)
    bareland_count = count(4,x)
    total = len(x)
    
    #apllying different rules
    if (background_count == total):
        return '0'
    elif (water_count >= 1 and water_count > 1.5 * grass_count and grass_count>0):
        return '2'
    elif (water_count >= 0.5 * total):
        return '2'
    elif (water_count >= 1 and water_count <= 1.5 * grass_count and grass_count>0):
        return '1'
    elif (water_count != 0 and grass_count == 0 ):
        return str(median)
    elif ( grass_count >=1 ):
        return '1'
    elif((bareland_count > buildings_count) and grass_count==0 and water_count==0 ):
        return '4'
    else:
        return '3'        

In [17]:

# Read all the different predictions of a district.

############### Change district, year, month and resolution accordingly  ################# 
years=['2016','2017','2018','2019']

for district in cities:
    print(district)
    main_folder=district     #we have stored monthly predections in a folder named by district name
    os.makedirs(main_folder+"/final",exist_ok=True)
    for year in years:
        print(year)
        Green_pixs=[]
        for infile in os.listdir(main_folder+"/pngs/"):
            if 'Sentinel_SR_' in infile and year in infile and 'median' not in infile: #reading all monthly predictions
                im = Image.open(main_folder+"/pngs/"+infile)
                month=infile[-6:-4]
                Green_pixs.append((month,np.unique(im,return_counts=True)[1][1]))
        print(Green_pixs)
        Green_pixs.sort(key=lambda x:x[1],reverse=True)
        print(Green_pixs)
        months=[Green_pixs[i][0] for i in range(min(5,len(Green_pixs)))] #taking best 5 months on the basis of greenery
        print(months)
        ##########################################################################################

        prediction1 = 'Sentinel_SR_'+ district +'_'+ year +'_' + months[0] + '.png'
        prediction2 = 'Sentinel_SR_'+ district +'_'+ year +'_' + months[1] + '.png'
        prediction3 = 'Sentinel_SR_'+ district +'_'+ year +'_' + months[2] + '.png'
        prediction4 = 'Sentinel_SR_'+ district +'_'+ year +'_' + months[3] + '.png'
        prediction5 = 'Sentinel_SR_'+ district +'_'+ year +'_' + months[4] + '.png'
        
        
        #defining median file name
        prediction_median = 'Sentinel_SR_Gurgaon_'+year+'_year_median.png'




        prediction1_path = main_folder+'/pngs/'+prediction1
        prediction2_path = main_folder+'/pngs/'+prediction2
        prediction3_path = main_folder+'/pngs/'+prediction3
        prediction4_path = main_folder+'/pngs/'+prediction4
        prediction5_path = main_folder+'/pngs/'+prediction5

        prediction_median_path = main_folder+'/pngs/'+prediction_median




        print(prediction1_path)
        print(prediction2_path)
        print(prediction3_path)
        print(prediction4_path)
        print(prediction5_path)
        print(prediction_median_path)

# Read all the required predictions

        prediction1 = imageio.imread(prediction1_path)
        prediction2 = imageio.imread(prediction2_path)
        prediction3 = imageio.imread(prediction3_path)
        prediction4 = imageio.imread(prediction4_path)
        prediction5 = imageio.imread(prediction5_path)

        prediction_median = imageio.imread(prediction_median_path)


        print("prediction1 ",np.unique(prediction1,return_counts=True))
        print("prediction2 ",np.unique(prediction2,return_counts=True))
        print("prediction3 ",np.unique(prediction3,return_counts=True))
        print("prediction4 ",np.unique(prediction4,return_counts=True))
        print("prediction5 ",np.unique(prediction5,return_counts=True))

        print("prediction_median ",np.unique(prediction_median,return_counts=True))

        image_dimension = prediction1.shape
        print(image_dimension)

        final_prediction = np.arange(image_dimension[0] * image_dimension[1]).reshape(image_dimension)
        print(final_prediction)


        for i in range(image_dimension[0]):
            for j in range(image_dimension[1]):
                x=[ prediction1[i][j], prediction2[i][j], prediction3[i][j], prediction4[i][j], prediction5[i][j]]
                final_prediction[i,j] = merge_prediction(x, prediction_median[i][j] )

        print("final_prediction ",np.unique(final_prediction,return_counts=True))

        imageio.imwrite(main_folder+'/final/'+district +'_prediction_'+year+'.png', final_prediction[:,:])


Gurgaon
2014
[('05', 272659), ('11', 900784), ('03', 1071409), ('01', 801385), ('07', 356010), ('09', 762246)]
[('03', 1071409), ('11', 900784), ('01', 801385), ('09', 762246), ('07', 356010), ('05', 272659)]
['03', '11', '01', '09', '07']
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2014_03.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2014_11.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2014_01.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2014_09.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2014_07.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2014_year_median.png
prediction1  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1141448, 1071409,   13939,  152251,  395758]))
prediction2  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1142867,  900784,   32044,  487081,  212029]))
prediction3  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1531997,  801385,   24403,  271956,  145064]))
prediction4  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1141448,

Lossy conversion from int64 to uint8. Range [0, 4]. Convert image to uint8 prior to saving to suppress this warning.


final_prediction  (array([0, 1, 2, 3, 4]), array([1141448, 1453915,   12106,   83973,   83363]))
2015
[('01', 639152), ('09', 1125361), ('05', 525888), ('03', 1147449), ('11', 1043231)]
[('03', 1147449), ('09', 1125361), ('11', 1043231), ('01', 639152), ('05', 525888)]
['03', '09', '11', '01', '05']
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2015_03.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2015_09.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2015_11.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2015_01.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2015_05.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2015_year_median.png
prediction1  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1141448, 1147449,   24969,  139450,  321489]))
prediction2  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1141448, 1125361,   19729,  138866,  349401]))
prediction3  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1150966, 1043231,   33352,  407614,  139642]))
predic

Lossy conversion from int64 to uint8. Range [0, 4]. Convert image to uint8 prior to saving to suppress this warning.


final_prediction  (array([0, 1, 2, 3, 4]), array([1141448, 1523542,   10951,   77568,   21296]))
2016
[('03', 1221061), ('07', 1381194), ('05', 471665), ('01', 1146760), ('09', 591327), ('11', 1172802)]
[('07', 1381194), ('03', 1221061), ('11', 1172802), ('01', 1146760), ('09', 591327), ('05', 471665)]
['07', '03', '11', '01', '09']
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2016_07.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2016_03.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2016_11.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2016_01.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2016_09.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2016_year_median.png
prediction1  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1181291, 1381194,   19991,   82502,  109827]))
prediction2  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1145478, 1221061,   13514,  199540,  195212]))
prediction3  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1151017, 1172802,  

...

final_prediction  (array([0, 1, 2, 3, 4]), array([1141448, 1517296,   18306,   77067,   20688]))
2018
[('05', 402754), ('03', 796309), ('01', 1184845), ('09', 939562), ('11', 1129576)]
[('01', 1184845), ('11', 1129576), ('09', 939562), ('03', 796309), ('05', 402754)]
['01', '11', '09', '03', '05']
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2018_01.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2018_11.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2018_09.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2018_03.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2018_05.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2018_year_median.png
prediction1  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1152185, 1184845,   44441,  245474,  147860]))
prediction2  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1164329, 1129576,   55201,  300014,  125685]))
prediction3  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1141597,  939562,   42946,  205502,  445198]))
predicti

Lossy conversion from int64 to uint8. Range [0, 4]. Convert image to uint8 prior to saving to suppress this warning.


final_prediction  (array([0, 1, 2, 3, 4]), array([1141448, 1519474,   23183,   72161,   18539]))
2019
[('05', 215853), ('01', 1184836), ('03', 1243605), ('07', 81767), ('11', 1119718), ('09', 1202581)]
[('03', 1243605), ('09', 1202581), ('01', 1184836), ('11', 1119718), ('05', 215853), ('07', 81767)]
['03', '09', '01', '11', '05']
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2019_03.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2019_09.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2019_01.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2019_11.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2019_05.png
Hariom_Landsat_new2-min/pngs/Landsat8_SR_Gurgaon_2019_year_median.png
prediction1  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1141448, 1243605,   19142,  180031,  190579]))
prediction2  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1142007, 1202581,   44539,  192602,  193076]))
prediction3  (Array([0, 1, 2, 3, 4], dtype=uint16), array([1150556, 1184836,   4

Lossy conversion from int64 to uint8. Range [0, 4]. Convert image to uint8 prior to saving to suppress this warning.


final_prediction  (array([0, 1, 2, 3, 4]), array([1141448, 1513497,   22865,   87004,    9991]))


In [228]:
#make final predictions colorful

from PIL import Image

district='Chennai'
year='2016'
image= '_new/' +  'final_prediction_results/'+resolution+district+'/'+district +'_prediction_'+year+'.png'


img = Image.open(image)
img = img.convert("RGBA")
pixdata = img.load()
print(img.getcolors())
for y in range(img.size[1]):
    for x in range(img.size[0]):
        if pixdata[x, y] == (0, 0, 0, 255): #background
            pixdata[x, y] = (0,0,0, 0)
        elif pixdata[x, y] == (64, 64, 64, 255):      #green
            pixdata[x, y] = (34,139,34, 255)
        elif pixdata[x, y] == (127, 127, 127, 255):    #water
            pixdata[x, y] = (2, 4, 251, 255)
        elif pixdata[x, y] == (191, 191, 191, 255):   #buildingd 
            pixdata[x, y] = (255, 255, 102, 255)
        elif pixdata[x, y] == (255, 255, 255, 255):   #bareland
            pixdata[x, y] = (255, 80, 80, 255)
img.save('_new/final_prediction_results/'+resolution+district+'/'+district+'_prediction_'+year+'_colored_median2.png')

[(9926, (255, 255, 255, 255)), (108347, (191, 191, 191, 255)), (4173, (127, 127, 127, 255)), (102763, (64, 64, 64, 255)), (117523, (0, 0, 0, 255))]
